In [2]:
import pandas as pd
import numpy as np

import os, glob

In [3]:
path = os.path.join(os.path.dirname(os.getcwd()), '../Data/Extracted_data/')
os.chdir(path)

In [4]:
#Fracture toughness
FT = pd.read_csv('fracture_toughness.csv')
FT.set_index('ID', inplace=True)
FT.drop(['Type', 'Percent_cold_work','KIC_stress_intensity_max_ksi_x_in1_2', 'KIC_stress_intensity_range_ksi_x_in1_2'], axis=1, inplace=True)

#TEP (both)
TEP = pd.read_csv('TEP_magnetic_susceptibility_summary.csv')
TEP.set_index('ID', inplace=True)
TEP.drop(['material', 'cold_work_percent'], axis=1, inplace=True)

#Nonlinear
UT_NL = pd.read_csv('UT_nonlinear.csv')
UT_NL.set_index('ID', inplace=True)
UT_NL.drop(['material', 'cold_work_percent'], axis=1, inplace=True)
UT_NL = UT_NL.groupby('ID').mean().add_suffix('_avg')

In [9]:
#Internal Friction 225MHz
IF_225_dfs=[]
for file in glob.glob('*friction*25MHz.csv'):
    df = pd.read_csv(file)
    IF_225_dfs.append(df)

UT_IF_225 = pd.concat(IF_225_dfs, axis=0, ignore_index=False)
UT_IF_225['Sample ID'] = UT_IF_225['Sample ID'].str.replace('%|CW|\,', '').str.strip().str.replace(' - ', '-')\
.str.replace('  ', ' ').str.replace(' ', '-').str.strip('-1')+ '-1'
UT_IF_225.drop(['% CW'], axis=1, inplace=True)
UT_IF_225.rename(columns={'Sample ID':'ID'}, inplace=True)
UT_IF_225.set_index('ID', inplace=True)
UT_IF_225 = UT_IF_225.add_suffix('_2.25MHz')

In [10]:
#Internal Friction 35MHz
IF_35_dfs=[]
for file in glob.glob('*friction*_5MHz.csv'):
    df = pd.read_csv(file)
    IF_35_dfs.append(df)

UT_IF_35 = pd.concat(IF_35_dfs, axis=0, ignore_index=False)
UT_IF_35['Sample ID'] = UT_IF_35['Sample ID'].str.replace('%|CW|\,', '').str.strip().str.replace(' - ', '-')\
.str.replace('  ', ' ').str.replace(' ', '-').str.strip('-1')+ '-1'
UT_IF_35.drop(['% CW'], axis=1, inplace=True)
UT_IF_35.rename(columns={'Sample ID':'ID'}, inplace=True)
UT_IF_35.set_index('ID', inplace=True)
UT_IF_35 = UT_IF_35.add_suffix('_3.5MHz')

In [11]:
#Backscattering
UT_BS = pd.read_csv('UT_backscattering.csv')

UT_BS['Sample ID'] = UT_BS['Sample ID'].str.replace('%|CW|\,', '').str.strip().str.replace(' - ', '-')\
.str.replace('  ', ' ').str.replace(' ', '-').str.strip('-1')+ '-1'
UT_BS.rename(columns={'Sample ID':'ID'}, inplace=True)
UT_BS.set_index('ID', inplace=True)

In [14]:
merged = FT.join([TEP, UT_NL, UT_IF_225, UT_IF_35, UT_BS], how='left')

In [20]:
merged.to_csv(os.path.join(os.path.dirname(os.getcwd()), '../Data/Merged_data/MERGE_FT_TEP_UT_on_ID.csv'))

In [19]:
merged

,JQ_lb-in_in2,KJIC_ksi_x_in1_2,magnetic_susceptibility_average,magnetic_susceptibility_neg_error,magnetic_susceptibility_pos_error,TEP_average,TEP_error,A2/A1^2_avg,Thickness (mm)_avg,R^2_avg,Beta_avg,% change from 0% CW_2.25MHz,Standard Error_2.25MHz,Standard Error (%)_2.25MHz,% change from 0% CW_3.5MHz,Standard Error_3.5MHz,Standard Error (%)_3.5MHz,% change from 0% CW
ID,,,,,,,,,,,,,,,,,,
304-0-1,730.0,147.2,NaN,NaN,NaN,NaN,NaN,0.034973,12.75,0.990,0.002743,0%,0.013,2.9%,0%,0.005,1.3%,0%
304-0-2,795.4,153.6,0.12,0.035,0.086,-1.547,0.049,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304-0-3,745.0,148.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304-0-4,1107.0,181.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304-0-5,1083.8,179.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A286-80-1,80.1,51.0,NaN,NaN,NaN,NaN,NaN,0.027902,12.76,0.996,0.002187,501%,0.025,2.0%,759%,0.022,1.5%,-64%
A286-80-2,78.5,50.5,0.01,0.002,0.002,-1.466,0.047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A286-80-3,87.0,53.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
